In [158]:
import pandas as pd
import elasticsearch
from elasticsearch_dsl import Search, A
from elasticsearch_dsl.query import MatchAll, Match

In [ ]:
multiterm aggregation
#https://www.elastic.co/guide/en/elasticsearch/reference/current/search-aggregations-bucket-terms-aggregation.html

In [2]:
es = elasticsearch.Elasticsearch(hosts=[{'host': 'localhost', 'port': 9200}])

In [3]:
query = {
    "query": {
        "bool": {
            "must": [
                {
                    "multi_match": {
                        "query": "",
                        "fuzziness": "AUTO",
                        "fields": [
                            "phrase",#,#,
                            "phrase.ext"
                            #"phrase.bigrams"
                            #"phrase.ngram"
                        ]
                    }
                },
            ],
            "should": [
                {
                    "multi_match": {
                        "query": "",
                        "fuzziness": 1,
                        "fields": [
                            "norm_phrase"
                            #"norm_phrase.bigrams"
                            #"norm_phrase.ngram"
                        ]
                    }
                }
            ]
        }
    },
    "size": 25,
    "sort": {
        "_score": {
            "order": "desc"
        }
    },
}

query = {
    "query": {
        "match_all": {}
    }
}
res = es.search(index="rdrs110722", body = query)

/tmp/ipykernel_11825/3176541370.py:46: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res = es.search(index="rdrs110722", body = query)


In [163]:
s = Search(using=es, index="rdrs110722").source(["Drugname_kw", "Diseasename_kw", "review_url_kw"]).query("match_all")

In [156]:
s = Search(using=es, index="rdrs110722").source(["Drugname_kw", "Diseasename_kw", "review_url_kw"]).aggs.bucket("UniqueDrugnames", agg_type="terms", field="Drugname_kw")

In [164]:
response = s.execute()

In [165]:
listed = []
#for hit in s[0:3800]:
for hit in s.scan():
    listed.append(hit.to_dict())

In [148]:
res = s.aggs.bucket("UniqueDrugnames", agg_type="terms", field="Drugname_kw").bucket("UniqueDrugnames", agg_type="sum", field="Diseasename_kw"


In [170]:
s = Search(using=es, index="rdrs110722").source(["Drugname_kw", "Diseasename_kw", "review_url_kw"]).query("match_all")
s.aggs.bucket("drugs", A("cardinality", field="Drugname_kw"))
res = s.execute()

RequestError: RequestError(400, 'search_phase_execution_exception', 'Text fields are not optimised for operations that require per-document field data like aggregations and sorting, so these operations are disabled by default. Please use a keyword field instead. Alternatively, set fielddata=true on [Drugname_kw] in order to load field data by uninverting the inverted index. Note that this can use significant memory.')

In [123]:
s["UniqueDrugnames"].bucket("tags_per_category")

TypeError: '<' not supported between instances of 'str' and 'int'

In [166]:
pd.DataFrame(listed)

,Drugname_kw,Diseasename_kw
0,[Амиксин],"[герпеса, Герпес]"
1,"[Иммунофлазид, Иммунофлазид]","[ГРВИ, грипа]"
2,"[Гавискон, Гавискон, Гавискон]",NaN
3,[Феназепам],NaN
4,[Виферон],[простуд]
...,...,...
3816,"[Кленбутерол, кленбутерол]",NaN
3817,"[Леденцы Вулкан Лакричные, леденцах лакричных,...",NaN
3818,"[Диафлекс РОМФАРМ, Диафлекс, омепрозол, Диафле...",NaN
3819,"[Целебрекс, цереблекс]",[обострении радикулита]
